# ASHE Table 7

-----

### Requirements

"Annual Summary of Hours and Earnings"


#### Observations & Dimensions

The `observations` are the numbers in the percentile columns.

The required dimensions are:

* **Geography** - in the `Code` column, one letter followed by 8 digits
* **Percentiles** - 10,20,30, etc
* **Time** - year, 4 digits
* **Gender** - Male, Female, All
* **Working Pattern** - Full time, Part time, All
* **Statistics** - The "topic" of the dataset, i.e "monthly pay net etc", in the filename

-----
    
Notes:

The "statistics" seems pointless because we're looking at one file. In production there are 24 per year per ASHE table.

It's always worth getting the file out of /sources and having a look over.

In [3]:
%cd mock-transformations/

/workspace/mock-transformations


In [4]:
from databaker.framework import *
import pandas as pd

tabs = loadxlstabs(r"""./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls""")

Loading ./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls which has size 846336 bytes
Table names: ['Notes', 'All', 'Male', 'Female', 'Full-Time', 'Part-Time', 'Male Full-Time', 'Male Part-Time', 'Female Full-Time', 'Female Part-Time']


In [ ]:
for tab in tabs:
    if tab.name == 'Notes':
        continue

    # Geography is easy
    geography = tab.excel_ref("B6").expand(DOWN).is_not_blank()
    
    # Manually naming 
